# Ensemble Modeling

## Imports

In [1]:
# Basic imports
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from statistics import mean

# Torch imports
import torch
import torchvision
import torch.optim as optim
from torch.nn import init
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

# Custom functions
from processing_functions_2 import *

## GPU Setup

In [2]:
# check if GPU available and assign it
torch.cuda.is_available()
device = torch.device('cuda', 1)
device

device(type='cuda', index=1)

## Modeling Setup

### MultiLabel

In [3]:
from torchvision import models as models

def model_multi_label_arch(pretrained, requires_grad):
    model = models.resnet50(progress=True, pretrained=pretrained)
    # to freeze the hidden layers
    if requires_grad == False:
        for param in model.parameters():
            param.requires_grad = False
    # to train the hidden layers
    elif requires_grad == True:
        for param in model.parameters():
            param.requires_grad = True
    # make the classification layer learnable
    model.fc = nn.Linear(2048, 8)
    return model

### MultiClass

In [4]:
class BasicConv(nn.Module):
    # initialize class
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, 
                 padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        # def conv layer, in_planes/out_planes = size of features
        self.conv = nn.Conv2d(in_planes, out_planes, 
                              kernel_size=kernel_size, stride=stride, 
                              padding=padding, dilation=dilation,
                              groups=groups, bias=bias)
        # batch normalization - normalization of the layers' inputs by re-centering and re-scaling
        self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) if bn else None
        # ReLu activation
        self.relu = nn.ReLU() if relu else None

    # create feed-forward network for conv layer
    def forward(self, x):
        x = self.conv(x)           # only 1 layer
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x
 
    
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1) # reshape to flatten tensor which is necessary in order to pass data into a linear layer
                                     # add -1
                                     # no flatten function in pytorch so need to create it

            
class ChannelGate(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
        super(ChannelGate, self).__init__()
        self.gate_channels = gate_channels
        self.mlp = nn.Sequential(
            Flatten(),
            nn.Linear(gate_channels, gate_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(gate_channels // reduction_ratio, gate_channels)
            )
        self.pool_types = pool_types
        
    def forward(self, x):
        channel_att_sum = None
        for pool_type in self.pool_types:
            if pool_type=='avg':
                avg_pool = F.avg_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( avg_pool )
            elif pool_type=='max':
                max_pool = F.max_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( max_pool )
            elif pool_type=='lp':
                lp_pool = F.lp_pool2d( x, 2, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( lp_pool )
            elif pool_type=='lse':
                # LSE pool only
                lse_pool = logsumexp_2d(x)
                channel_att_raw = self.mlp( lse_pool )

            if channel_att_sum is None:
                channel_att_sum = channel_att_raw
            else:
                channel_att_sum = channel_att_sum + channel_att_raw

        scale = F.sigmoid( channel_att_sum ).unsqueeze(2).unsqueeze(3).expand_as(x)
        return x * scale

    
def logsumexp_2d(tensor):
    tensor_flatten = tensor.view(tensor.size(0), tensor.size(1), -1)
    s, _ = torch.max(tensor_flatten, dim=2, keepdim=True)
    outputs = s + (tensor_flatten - s).exp().sum(dim=2, keepdim=True).log()
    return outputs


class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat( (torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )

       
class SpatialGate(nn.Module):
    def __init__(self):
        super(SpatialGate, self).__init__()
        kernel_size = 7
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)
    def forward(self, x):
        x_compress = self.compress(x)
        x_out = self.spatial(x_compress)
        scale = F.sigmoid(x_out) # broadcasting
        return x * scale

       
class CBAM(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max'], no_spatial=False):
        super(CBAM, self).__init__()
        self.ChannelGate = ChannelGate(gate_channels, reduction_ratio, pool_types)
        self.no_spatial=no_spatial
        if not no_spatial:
            self.SpatialGate = SpatialGate()
            
    def forward(self, x):
        x_out = self.ChannelGate(x)
        if not self.no_spatial:
            x_out = self.SpatialGate(x_out)
        return x_out
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, use_cbam=False):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

        if use_cbam:
            self.cbam = CBAM( planes, 16 )
        else:
            self.cbam = None

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        if not self.cbam is None:
            out = self.cbam(out)

        out += residual
        out = self.relu(out)

        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, use_cbam=False):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

        if use_cbam:
            self.cbam = CBAM( planes * 4, 16 )
        else:
            self.cbam = None

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        if not self.cbam is None:
            out = self.cbam(out)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, layers,  network_type, num_classes, att_type=None):
        self.inplanes = 64
        super(ResNet, self).__init__()

        self.network_type = network_type
        # different model config between ImageNet and CIFAR 
        if network_type == "ImageNet":
            self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
            self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
            self.avgpool = nn.AvgPool2d(7)
        else:
            self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)

        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        if att_type=='BAM':
            self.bam1 = BAM(64*block.expansion)
            self.bam2 = BAM(128*block.expansion)
            self.bam3 = BAM(256*block.expansion)
        else:
            self.bam1, self.bam2, self.bam3 = None, None, None

        self.layer1 = self._make_layer(block, 64,  layers[0], att_type=att_type)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, att_type=att_type)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, att_type=att_type)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, att_type=att_type)

        self.fc = nn.Linear(512 * block.expansion, num_classes)
        #self.fc = nn.Linear(512 * block.expansion, 1) # linear is output of probability distributions
        self.softmax = torch.nn.Sigmoid()
        init.kaiming_normal(self.fc.weight)
        for key in self.state_dict():
            if key.split('.')[-1]=="weight":
                if "conv" in key:
                    init.kaiming_normal(self.state_dict()[key], mode='fan_out')
                if "bn" in key:
                    if "SpatialGate" in key:
                        self.state_dict()[key][...] = 0
                    else:
                        self.state_dict()[key][...] = 1
            elif key.split(".")[-1]=='bias':
                self.state_dict()[key][...] = 0

    def _make_layer(self, block, planes, blocks, stride=1, att_type=None):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, use_cbam=att_type=='CBAM'))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, use_cbam=att_type=='CBAM'))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if self.network_type == "ImageNet":
            x = self.maxpool(x)

        x = self.layer1(x)
        if not self.bam1 is None:
            x = self.bam1(x)

        x = self.layer2(x)
        if not self.bam2 is None:
            x = self.bam2(x)

        x = self.layer3(x)
        if not self.bam3 is None:
            x = self.bam3(x)

        x = self.layer4(x)

        if self.network_type == "ImageNet":
            x = self.avgpool(x)
        else:
            x = F.avg_pool2d(x, 4)
            
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        #return F.normalize(x, dim=-1)
        return self.softmax(x)
    
def ResidualNet(network_type, depth, num_classes, att_type):

    assert network_type in ["ImageNet", "CIFAR10", "CIFAR100"], "network type should be ImageNet or CIFAR10 / CIFAR100"
    assert depth in [5, 18, 34, 50, 101], 'network depth should be 18, 34, 50 or 101'

    if depth == 18:
        model = ResNet(BasicBlock, [2, 2, 2, 2], network_type, num_classes, att_type)
        
    elif depth == 5:
        model = ResNet(BasicBlock, [1, 1, 2, 1], network_type, num_classes, att_type)

    elif depth == 34:
        model = ResNet(BasicBlock, [3, 4, 6, 3], network_type, num_classes, att_type)

    elif depth == 50:
        model = ResNet(Bottleneck, [3, 4, 6, 3], network_type, num_classes, att_type)

    elif depth == 101:
        model = ResNet(Bottleneck, [3, 4, 23, 3], network_type, num_classes, att_type)

    return model

## Load Trained Models
TODO: update to save full model instead

In [5]:
#intialize the model
model_multi_label = model_multi_label_arch(pretrained=True, requires_grad=False).to(device)

# load the model checkpoint
checkpoint = torch.load('model_multi_class_large_with_clahe_afterdrops_v1.pth')
# load model weights state_dict
model_multi_label.load_state_dict(checkpoint['model_state_dict'])
model_multi_label.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
model_multi_class_aug = ResidualNet('ImageNet', 34, 8, 'CBAM').to(device)

# load the model checkpoint
checkpoint = torch.load('model_1_clahe_aug_weights.pth')
# load model weights state_dict
model_multi_class_aug.load_state_dict(checkpoint)
model_multi_class_aug.eval()

C:\Users\dental-1\AppData\Local\Temp/ipykernel_17412/4190062143.py:232: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.fc.weight)
C:\Users\dental-1\AppData\Local\Temp/ipykernel_17412/4190062143.py:236: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.state_dict()[key], mode='fan_out')


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (avgpool): AvgPool2d(kernel_size=7, stride=7, padding=0)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (cbam): CBAM(
        (ChannelGate): ChannelGate(
          (mlp): Sequential(
            (0): Flatten()
            (1): Linear(in_features=64, out_features=4, bias=True)
          

In [7]:
model_multi_class_smote = ResidualNet('ImageNet', 5, 8, 'CBAM').to(device)

# load the model checkpoint
#checkpoint = torch.load('model_1_smote_weights_BEST.pth')
checkpoint = torch.load('model_1_smote_weights_FINAL_7.pth')
# checkpoint = torch.load('model_1_smote_weights_clahe_BEST_1111.pth')
# load model weights state_dict
model_multi_class_smote.load_state_dict(checkpoint)
model_multi_class_smote.eval()

C:\Users\dental-1\AppData\Local\Temp/ipykernel_17412/4190062143.py:232: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.fc.weight)
C:\Users\dental-1\AppData\Local\Temp/ipykernel_17412/4190062143.py:236: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.state_dict()[key], mode='fan_out')


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (avgpool): AvgPool2d(kernel_size=7, stride=7, padding=0)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (cbam): CBAM(
        (ChannelGate): ChannelGate(
          (mlp): Sequential(
            (0): Flatten()
            (1): Linear(in_features=64, out_features=4, bias=True)
          

In [8]:
models = [model_multi_class_aug, 
          model_multi_class_smote, 
          model_multi_label]

## Showing sample predictions

In [9]:
anomaly_key = {0: 'Filling',
               1: 'Root canal treatment',
               2: 'Implant',
               3: 'Caries - low risk',
               4: 'Caries - moderate risk',
               5: 'Caries - high risk',
               6: 'Periodontitis',
               7: 'Other'}

In [10]:
training_set = pd.read_csv('C:/Documents/Dental_Detection/data_csv/train_data_final.csv')
training_set.head()

,Unnamed: 0,Unnamed: 0.1,file_path,anomaly_codes
0,0,9473,C:/Documents/Dental_Detection/Segmented_Images...,7.0
1,1,12578,C:/Documents/Dental_Detection/Segmented_Images...,7.0
2,2,13525,C:/Documents/Dental_Detection/Segmented_Images...,7.0
3,3,11362,C:/Documents/Dental_Detection/Segmented_Images...,0.0
4,4,5455,C:/Documents/Dental_Detection/Segmented_Images...,7.0


In [11]:
training_set['anomaly_codes'].value_counts()

7.0    9033
0.0    1316
1.0     184
8.0     155
6.0      74
5.0      45
4.0      43
2.0      26
3.0      13
Name: anomaly_codes, dtype: int64

## Predictions with Images

In [12]:
# import matplotlib.pyplot as plt

# for index, row in validation_set.iterrows():
#     if index < 100:
#         for model in models:
#             # Get image
#             image_path = row['file_path']
#             img = cv2.imread(image_path)     # loads image from file
#             img = cv2.resize(img, (224, 224))        # resize image to 224x224
#             img = np.array(img, dtype=np.float32)    # change data type to float
#             img = (img / 255.)                       # normalize colors to be 0-1
#             img = img[:, :, (2, 1, 0)]               # reorder RGB
#             img = torchvision.transforms.ToTensor()(img)  # convert to tensor
#             target = row['anomaly_codes']

#             image = img.to(device).unsqueeze(0)
#             outputs = model(image)
#             outputs = torch.sigmoid(outputs)
#             outputs = outputs.detach().cpu()
#             sorted_indices = np.argsort(outputs[0])
#             best = sorted_indices[-3:].numpy()
#             string_predicted = ''
#             string_actual = ''
#             for i in range(len(best)):
#                 string_predicted += f"{anomaly_key[best[i]]} | "
#             string_actual += anomaly_key[int(target)]
#             image = image.squeeze(0)
#             image = image.detach().cpu().numpy()
#             image = np.transpose(image, (1, 2, 0))
#             plt.imshow(image)
#             plt.axis('off')
#             plt.title(f"PREDICTED: {string_predicted}\nACTUAL: {string_actual}")
#     #         plt.savefig(f"outputs_multi_label/inference_{counter}.jpg")
#             plt.show()

## Measuring overall performance

In [13]:
df_actuals = training_set['anomaly_codes']

#### Calculate original prediction probabilities for each model:

In [14]:
all_predictions = []
for i in range(len(models)):
    model = models[i]
    predictions = []

    for index, row in tqdm(training_set.iterrows(), total=training_set.shape[0]):
        image_path = row['file_path']
        img = cv2.imread(image_path)     # loads image from file
         # declaration of clahe
        clahe = cv2.createCLAHE(clipLimit=12.0, tileGridSize=(8,8))
        
        if((i == 0) | (i == 2)):
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = clahe.apply(img)
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

        img = cv2.resize(img, (224, 224))        # resize image to 224x224
        img = np.array(img, dtype=np.float32)    # change data type to float
        img = (img / 255.)                       # normalize colors to be 0-1
        img = img[:, :, (2, 1, 0)]               # reorder RGB
        img = torchvision.transforms.ToTensor()(img)  # convert to tensor
        target = row['anomaly_codes']

        image = img.to(device).unsqueeze(0)
        outputs = model(image)
        if i == 2:
            outputs = torch.sigmoid(outputs)
        outputs = outputs.detach().cpu()

        predictions += [outputs.squeeze().tolist()]
        df_predictions = pd.DataFrame(predictions)
    all_predictions += [df_predictions]

  0%|          | 0/10889 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\envs\dental_project_ajw\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 10889/10889 [03:56<00:00, 46.11it/s]


#### calculate optimal thresholds for each class in each model that maximizes the classes' F1 score

In [15]:
from sklearn.metrics import f1_score
import numpy as np

all_thresholds = []
df_actuals_dummied = pd.get_dummies(df_actuals.astype(int))

for i in range(len(models)):
    print("Model ", i)
    preds = all_predictions[i]
    final_thresholds = []
    
    for label in range(preds.shape[1]):
        thresholds = np.arange(0.0, 1.0, 0.0001) 
        f1_scores = []
        for threshold in tqdm(thresholds, total = len(thresholds)):
            y_pred = pd.Series(preds[label] > threshold).astype(int)
            f1_scores += [f1_score(df_actuals_dummied[label], y_pred, zero_division = 0)]
        max_f1 = max(f1_scores)
        max_threshold = thresholds[f1_scores.index(max_f1)]
        final_thresholds += [max_threshold]
        print("Class %d: Max F1_score of %.3f at a threshold value of %.5f" % (label, max_f1, max_threshold))
        
    all_thresholds += [final_thresholds]

Model  0


100%|██████████| 10000/10000 [00:25<00:00, 395.47it/s]


Class 0: Max F1_score of 0.872 at a threshold value of 0.00020


100%|██████████| 10000/10000 [00:22<00:00, 448.82it/s]


Class 1: Max F1_score of 0.875 at a threshold value of 0.98770


...

Class 5: Max F1_score of 0.073 at a threshold value of 0.04130


100%|██████████| 10000/10000 [00:19<00:00, 525.86it/s]


Class 6: Max F1_score of 0.092 at a threshold value of 0.05270


100%|██████████| 10000/10000 [00:23<00:00, 430.85it/s]

Class 7: Max F1_score of 0.911 at a threshold value of 0.34300


#### make final predictions based on new thresholds

In [16]:
all_predictions_final = []
for i in range(len(models)):
    thresh = all_thresholds[i]
    preds = all_predictions[i]
    
    preds_final = preds.copy()
    for label in range(preds.shape[1]):
        preds_final[label] = pd.Series(preds[label] > thresh[label]).astype(int)
    
    all_predictions_final += [preds_final]

In [28]:
def ensemble_weighted(preds, weights, vote_threshold = 3):
    predictions = weights[0]*preds[0] + weights[1]*preds[1] + weights[2]*preds[2]
    predictions = pd.DataFrame(predictions > vote_threshold).astype(int)
    
    predictions[7] = predictions[[0, 1, 2, 3, 4, 5, 6]].sum(axis = 1)
    predictions[7] = [0 if x > 0 else 1 for x in predictions[7]]
    
    return(predictions)

In [29]:
def print_evaluation_results(vote_results, actuals):
    results_df = pd.DataFrame()
    
    for col in vote_results.columns:
        y_pred = vote_results[col]
        y_true = actuals[col]
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
#         accuracy = accuracy_score(y_true, y_pred)
        results = {'Class': col, 'Precision': precision, 'Recall': recall, 'F1': f1}
        results_df = results_df.append(results, ignore_index = True)
#         print("Class %s: PRECISION %.2f, RECALL %.2f, F1 %.2f" % (col, precision, recall, f1))
#         print(confusion_matrix(y_true, y_pred))   
    return results_df

## Ensemble 1 - Majority Vote

#### Run Model

In [19]:
final = ensemble_weighted(all_predictions_final, [1, 1, 1], vote_threshold = 1)

#### Results

In [20]:
results = print_evaluation_results(vote_results = final, actuals = df_actuals_dummied)
results

,Class,Precision,Recall,F1
0,0.0,0.769501,0.832067,0.799562
1,1.0,0.744898,0.793478,0.768421
2,2.0,0.538462,0.807692,0.646154
3,3.0,0.444444,0.923077,0.600000
4,4.0,0.080808,0.372093,0.132780
5,5.0,0.119718,0.377778,0.181818
6,6.0,0.037260,0.418919,0.068433
7,7.0,0.950431,0.855419,0.900425


## Ensemble 2 - Weighted by Model

#### create list of possible weight combos 

In [32]:
weight1 = np.arange(0, 2, 0.1)
weight2 = np.arange(0, 2, 0.1)
weight3 = np.arange(0, 2, 0.1)

weights = []
for weight_1 in weight1:
    for weight_2 in weight2:
        for weight_3 in weight3:
            weights += [[weight_1, weight_2, weight_3]]

#### Use grid search to find optimal weight combo by maximizing the mean of evaluation metrics from classes 0,1,2
##### We know evaluation metrics are low for classes 2-6 and are less concerned about 7, so best to optimize weights based on performance of 0-2

In [12]:
def get_f1s(weighted_preds, actuals):
    f1s = []
    for col in weighted_preds.columns[0:3]:
        y_pred = weighted_preds[col]
        y_true = actuals[col]
        f1 = f1_score(y_true, y_pred)
        f1s += [f1]
    return(mean(f1s))

def get_recall(weighted_preds, actuals):
    recalls = []
    for col in weighted_preds.columns[0:3]:
        y_pred = weighted_preds[col]
        y_true = actuals[col]
        recall = recall_score(y_true, y_pred)
        recalls += [recall]
    return(mean(recalls))

def get_precision(weighted_preds, actuals):
    precisions = []
    for col in weighted_preds.columns[0:3]:
        y_pred = weighted_preds[col]
        y_true = actuals[col]
        precision = precision_score(y_true, y_pred)
        precisions += [precision]
    return(mean(precisions))

In [13]:
def grid_search(preds, actuals, weights):
    f1_list = []
    for test_weights in tqdm(weights, total = len(weights)):
        weighted_preds = ensemble_weighted(preds, test_weights)
        weighted_f1s = get_f1s(weighted_preds, actuals)
        f1_list += [weighted_f1s]
    return(f1_list)

def grid_search_recall(preds, actuals, weights):
    f1_list = []
    for test_weights in tqdm(weights, total = len(weights)):
        weighted_preds = ensemble_weighted(preds, test_weights)
        weighted_f1s = get_recall(weighted_preds, actuals)
        f1_list += [weighted_f1s]
    return(f1_list)

def grid_search_precision(preds, actuals, weights):
    f1_list = []
    for test_weights in tqdm(weights, total = len(weights)):
        weighted_preds = ensemble_weighted(preds, test_weights)
        weighted_f1s = get_precision(weighted_preds, actuals)
        f1_list += [weighted_f1s]
    return(f1_list)

In [14]:
def grid_search_model(preds, actuals, weights):
    f1_list = grid_search(preds, actuals, weights)
    f1_indexes = []
    bools = f1_list == max(f1_list)
    for indx in range(len(bools)):
        if(bools[indx] == True):
            f1_indexes.append(indx)
    
    weights = np.array(weights)
    filtered_weights = np.array(weights[f1_indexes])
    
    recall_list = grid_search_recall(preds, actuals, filtered_weights)
    recall_indexes = []
    bools = recall_list == max(recall_list)
    for indx in range(len(bools)):
        if(bools[indx] == True):
            recall_indexes.append(indx)
    
    filtered_weights = np.array(filtered_weights[recall_indexes])
    
    precision_list = grid_search_precision(preds, actuals, filtered_weights)
    
    final_weights = filtered_weights[precision_list.index(max(precision_list))]
    
    return(final_weights)

In [15]:
def get_metrics(weighted_preds, actuals):
    f1s = []
    recalls = []
    precisions = []
    for col in weighted_preds.columns[0:3]:
        y_pred = weighted_preds[col]
        y_true = actuals[col]
        
        f1 = f1_score(y_true, y_pred)
        f1s += [f1]
        
        recall = recall_score(y_true, y_pred)
        recalls += [recall]
        
        precision = precision_score(y_true, y_pred)
        precisions += [precision]
        
    return(mean(f1s), mean(recalls), mean(precisions))

In [16]:
def grid_search_az(preds, actuals, weights):
    f1_list = []
    for test_weights in tqdm(weights, total = len(weights)):
        weighted_preds = ensemble_weighted(preds, test_weights)
        [avg_f1, avg_recall, avg_precision] = get_metrics(weighted_preds, actuals)
        f1_list += [avg_f1]
    return(f1_list)

In [27]:
final_weights_2 = grid_search_model(all_predictions_final, df_actuals_dummied, weights)
final_final_2 = ensemble_weighted(all_predictions_final, final_weights_2)

100%|██████████| 466/466 [00:05<00:00, 84.81it/s]


In [28]:
final_weights_2

array([1.2, 1.9, 0. ])

In [29]:
results_2 = print_evaluation_results(vote_results = final_final_2, actuals = df_actuals_dummied)
results_2

,Class,Precision,Recall,F1
0,0.0,0.897533,0.718845,0.798312
1,1.0,0.835366,0.744565,0.787356
2,2.0,0.740741,0.769231,0.754717
3,3.0,0.500000,0.846154,0.628571
4,4.0,0.392857,0.255814,0.309859
5,5.0,0.185185,0.333333,0.238095
6,6.0,0.033419,0.351351,0.061033
7,7.0,0.928678,0.898041,0.913102


## Ensemble 3 - Weighted by Class

In [17]:
def grid_search_class(preds, actuals, weights):
    final_weight_list = []
    for tooth_class in range(8):
        f1_list = []
        for test_weights in tqdm(weights, total = len(weights)):
            predictions = test_weights[0]*preds[0][tooth_class] + test_weights[1]*preds[1][tooth_class] + test_weights[2]*preds[2][tooth_class]
            predictions = pd.Series(predictions > 3).astype(int)
            weighted_f1s = f1_score(predictions, actuals[tooth_class])
            f1_list += [weighted_f1s]
        f1_indexes = []
        bools = f1_list == max(f1_list)
        for indx in range(len(bools)):
            if(bools[indx] == True):
                f1_indexes.append(indx)
                
        weights = np.array(weights)
        filtered_weights = np.array(weights[f1_indexes])
        
        recall_list = []
        for test_weights in tqdm(filtered_weights, total = len(filtered_weights)):
            predictions = test_weights[0]*preds[0][tooth_class] + test_weights[1]*preds[1][tooth_class] + test_weights[2]*preds[2][tooth_class]
            predictions = pd.Series(predictions > 3).astype(int)
            weighted_recalls = recall_score(predictions, actuals[tooth_class])
            recall_list += [weighted_recalls]
        recall_indexes = []
        bools = recall_list == max(recall_list)
        for indx in range(len(bools)):
            if(bools[indx] == True):
                recall_indexes.append(indx)
                
        filtered_weights = np.array(filtered_weights[recall_indexes])
        
        precision_list = []
        for test_weights in tqdm(filtered_weights, total = len(filtered_weights)):
            predictions = test_weights[0]*preds[0][tooth_class] + test_weights[1]*preds[1][tooth_class] + test_weights[2]*preds[2][tooth_class]
            predictions = pd.Series(predictions > 3).astype(int)
            weighted_precision = precision_score(predictions, actuals[tooth_class])
            precision_list += [weighted_precision]
        
        final_weight_list.append(filtered_weights[precision_list.index(max(precision_list))])
    return(final_weight_list)

In [18]:
def ensemble_class(preds, weights):
    class_predictions = []
    predictions_df = pd.DataFrame()

    for tooth_class in preds[0].columns:
            predictions = weights.iloc[tooth_class,0]*preds[0][tooth_class] + weights.iloc[tooth_class,1]*preds[1][tooth_class] + weights.iloc[tooth_class,2]*preds[2][tooth_class]
            predictions = pd.Series(predictions > 3).astype(int)

            class_predictions.append(predictions)
              
    predictions_df[0] = class_predictions[0]
    predictions_df[1] = class_predictions[1]
    predictions_df[2] = class_predictions[2]
    predictions_df[3] = class_predictions[3]
    predictions_df[4] = class_predictions[4]
    predictions_df[5] = class_predictions[5]
    predictions_df[6] = class_predictions[6]
    predictions_df[7] = class_predictions[7]
        
    
    predictions_df[7] = predictions_df[[0, 1, 2, 3, 4, 5, 6]].sum(axis = 1)
    predictions_df[7] = [0 if x > 0 else 1 for x in predictions_df[7]]     

    return(predictions_df)

#### Calculate optimal class weights

In [32]:
final_weight_list = grid_search_class(all_predictions_final, df_actuals_dummied, weights)

100%|██████████| 2222/2222 [00:05<00:00, 373.87it/s]


In [33]:
final_weight_list

[array([1.6, 1.5, 1.5]),
 array([1.6, 1.5, 1.5]),
 array([1.2, 1.9, 0. ]),
 array([1.2, 1.9, 0. ]),
 array([1.2, 1.9, 0. ]),
 array([1.2, 1.9, 0. ]),
 array([1.2, 0. , 1.9]),
 array([0.1, 1.1, 1.9])]

#### Run Model

In [34]:
final_class_ensemble = ensemble_class(all_predictions_final, pd.DataFrame(final_weight_list))

#### Results

In [35]:
results_3 = print_evaluation_results(vote_results = final_class_ensemble, actuals = df_actuals_dummied)
results_3

,Class,Precision,Recall,F1
0,0.0,0.881013,0.793313,0.834866
1,1.0,0.820225,0.793478,0.806630
2,2.0,0.740741,0.769231,0.754717
3,3.0,0.500000,0.846154,0.628571
4,4.0,0.392857,0.255814,0.309859
5,5.0,0.185185,0.333333,0.238095
6,6.0,0.107143,0.081081,0.092308
7,7.0,0.943366,0.973652,0.958270


## Running new weights on validation

In [19]:
validation_set = pd.read_csv('C:/Documents/Dental_Detection/data_csv/valid_data_pano0_6.csv')
validation_set.head()

,Unnamed: 0,file_path,anomaly_codes
0,8309,C:/Documents/Dental_Detection/Segmented_Images...,7.0
1,7245,C:/Documents/Dental_Detection/Segmented_Images...,7.0
2,2786,C:/Documents/Dental_Detection/Segmented_Images...,7.0
3,8441,C:/Documents/Dental_Detection/Segmented_Images...,0.0
4,3973,C:/Documents/Dental_Detection/Segmented_Images...,7.0


In [20]:
validation_set['anomaly_codes'].value_counts()

7.0    2258
0.0     344
1.0      51
8.0      39
6.0      27
5.0      15
4.0      14
2.0       8
3.0       3
Name: anomaly_codes, dtype: int64

In [21]:
df_actuals_valid = validation_set['anomaly_codes']
df_actuals_dummied_valid = pd.get_dummies(df_actuals_valid.astype(int))

In [22]:
all_predictions_valid = []
for i in range(len(models)):
    model = models[i]
    predictions = []

    for index, row in tqdm(validation_set.iterrows(), total = validation_set.shape[0]):
        image_path = row['file_path']
        img = cv2.imread(image_path)     # loads image from file
         # declaration of clahe
        clahe = cv2.createCLAHE(clipLimit=12.0, tileGridSize=(8,8))
        
        if((i == 0) | (i == 2)):
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = clahe.apply(img)
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

        img = cv2.resize(img, (224, 224))        # resize image to 224x224
        img = np.array(img, dtype=np.float32)    # change data type to float
        img = (img / 255.)                       # normalize colors to be 0-1
        img = img[:, :, (2, 1, 0)]               # reorder RGB
        img = torchvision.transforms.ToTensor()(img)  # convert to tensor
        target = row['anomaly_codes']

        image = img.to(device).unsqueeze(0)
        outputs = model(image)
        if i == 2:
            outputs = torch.sigmoid(outputs)
        outputs = outputs.detach().cpu()

        predictions += [outputs.squeeze().tolist()]
        df_predictions = pd.DataFrame(predictions)
    all_predictions_valid += [df_predictions]

  0%|          | 0/2759 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\envs\dental_project_av\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 2759/2759 [00:46<00:00, 59.32it/s]


In [23]:
from sklearn.metrics import f1_score
import numpy as np

all_thresholds = []


for i in range(len(models)):
    print("Model ", i)
    preds = all_predictions_valid[i]
    final_thresholds = []
    
    for label in range(preds.shape[1]):
        thresholds = np.arange(0.0, 1.0, 0.0001) 
        f1_scores = []
        for threshold in tqdm(thresholds, total = len(thresholds)):
            y_pred = pd.Series(preds[label] > threshold).astype(int)
            f1_scores += [f1_score(df_actuals_dummied_valid[label], y_pred, zero_division = 0)]
        max_f1 = max(f1_scores)
        max_threshold = thresholds[f1_scores.index(max_f1)]
        final_thresholds += [max_threshold]
        print("Class %d: Max F1_score of %.3f at a threshold value of %.5f" % (label, max_f1, max_threshold))
        
    all_thresholds += [final_thresholds]

Model  0


100%|██████████| 10000/10000 [00:11<00:00, 884.86it/s]


Class 0: Max F1_score of 0.627 at a threshold value of 0.00010


100%|██████████| 10000/10000 [00:10<00:00, 932.97it/s]


Class 1: Max F1_score of 0.786 at a threshold value of 0.18180


...

Class 5: Max F1_score of 0.214 at a threshold value of 0.04880


100%|██████████| 10000/10000 [00:10<00:00, 945.44it/s]


Class 6: Max F1_score of 0.088 at a threshold value of 0.02810


100%|██████████| 10000/10000 [00:11<00:00, 855.51it/s]

Class 7: Max F1_score of 0.906 at a threshold value of 0.40400


In [25]:
all_thresholds

[[0.0001, 0.18180000000000002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.999],
 [0.8727, 0.8496, 0.3764, 0.7791, 0.9775, 0.9851000000000001, 0.007, 0.0],
 [0.2452,
  0.5074000000000001,
  0.4101,
  0.041100000000000005,
  0.055200000000000006,
  0.0488,
  0.0281,
  0.404]]

In [40]:
all_thresholds

[[0.0002, 0.9877, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9992000000000001],
 [0.8023,
  0.8919,
  0.9062,
  0.9773000000000001,
  0.9994000000000001,
  0.9995,
  0.8550000000000001,
  0.0],
 [0.2116,
  0.6755,
  0.42460000000000003,
  0.048600000000000004,
  0.0316,
  0.0413,
  0.052700000000000004,
  0.343]]

In [26]:
all_predictions_final_valid = []
for i in range(len(models)):
    thresh = all_thresholds[i]
    preds = all_predictions_valid[i]
    
    preds_final = preds.copy()
    for label in range(preds.shape[1]):
        preds_final[label] = pd.Series(preds[label] > thresh[label]).astype(int)
    
    all_predictions_final_valid += [preds_final]

In [42]:
all_predictions_final_valid_v2 = []
for i in range(len(models)):
    preds = all_predictions_valid[i]
    
    preds_final = preds.copy()
    for label in range(preds.shape[1]):
        preds_final[label] = pd.Series(preds[label] > .5).astype(int)
    
    all_predictions_final_valid_v2 += [preds_final]

### Ensemble 1 Results - Validation

In [30]:
final_valid = ensemble_weighted(all_predictions_final_valid, [1, 1, 1], vote_threshold = 1)
results_1_valid = print_evaluation_results(vote_results = final_valid, actuals = df_actuals_dummied_valid)
results_1_valid

,Class,Precision,Recall,F1
0,0.0,0.700658,0.619186,0.657407
1,1.0,0.587302,0.725490,0.649123
2,2.0,0.181818,0.500000,0.266667
3,3.0,0.031250,0.666667,0.059701
4,4.0,0.021930,0.357143,0.041322
5,5.0,0.025210,0.400000,0.047431
6,6.0,0.009834,0.703704,0.019398
7,7.0,0.852097,0.170948,0.284766


In [44]:
final_valid = ensemble_weighted(all_predictions_final_valid_v2, [1, 1, 1], vote_threshold = 1)
results_1_valid = print_evaluation_results(vote_results = final_valid, actuals = df_actuals_dummied_valid)
results_1_valid

C:\ProgramData\Anaconda3\envs\dental_project_ajw\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\envs\dental_project_ajw\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\envs\dental_project_ajw\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\envs\dental_project_a

,Class,Precision,Recall,F1
0,0.0,0.779006,0.409884,0.537143
1,1.0,0.588235,0.784314,0.672269
2,2.0,0.000000,0.000000,0.000000
3,3.0,0.000000,0.000000,0.000000
4,4.0,0.000000,0.000000,0.000000
5,5.0,0.000000,0.000000,0.000000
6,6.0,0.000000,0.000000,0.000000
7,7.0,0.881673,0.980071,0.928272


### Ensemble 2 Results - Validation

In [33]:
final_weights_valid = grid_search_model(all_predictions_final_valid, df_actuals_dummied_valid, weights)

100%|██████████| 50/50 [00:00<00:00, 152.38it/s]


In [34]:
final_2_valid = ensemble_weighted(all_predictions_final_valid, final_weights_valid)
results_2_valid = print_evaluation_results(vote_results = final_2_valid, actuals = df_actuals_dummied_valid)
results_2_valid

,Class,Precision,Recall,F1
0,0.0,0.703252,0.502907,0.586441
1,1.0,0.545455,0.470588,0.505263
2,2.0,1.000000,0.375000,0.545455
3,3.0,1.000000,0.333333,0.500000
4,4.0,0.333333,0.071429,0.117647
5,5.0,0.230769,0.200000,0.214286
6,6.0,0.054545,0.222222,0.087591
7,7.0,0.890249,0.934012,0.911606


In [46]:
final_2_valid = ensemble_weighted(all_predictions_final_valid_v2, final_weights_2)
results_2_valid = print_evaluation_results(vote_results = final_2_valid, actuals = df_actuals_dummied_valid)
results_2_valid

C:\ProgramData\Anaconda3\envs\dental_project_ajw\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\envs\dental_project_ajw\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\envs\dental_project_ajw\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\envs\dental_project_a

,Class,Precision,Recall,F1
0,0.0,0.784810,0.360465,0.494024
1,1.0,0.693878,0.666667,0.680000
2,2.0,0.000000,0.000000,0.000000
3,3.0,0.000000,0.000000,0.000000
4,4.0,0.000000,0.000000,0.000000
5,5.0,0.000000,0.000000,0.000000
6,6.0,0.000000,0.000000,0.000000
7,7.0,0.871865,0.985385,0.925156


### Ensemble 3 Results - Validation

In [35]:
final_weight_list = grid_search_class(all_predictions_final_valid, df_actuals_dummied_valid, weights)

100%|██████████| 2222/2222 [00:03<00:00, 644.94it/s]


In [36]:
final_class_ensemble_valid = ensemble_class(all_predictions_final_valid, pd.DataFrame(final_weight_list))
results_3_valid = print_evaluation_results(vote_results = final_class_ensemble_valid, actuals = df_actuals_dummied_valid)
results_3_valid

,Class,Precision,Recall,F1
0,0.0,0.700658,0.619186,0.657407
1,1.0,0.705882,0.705882,0.705882
2,2.0,1.000000,0.375000,0.545455
3,3.0,1.000000,0.333333,0.500000
4,4.0,0.333333,0.071429,0.117647
5,5.0,0.230769,0.200000,0.214286
6,6.0,0.081081,0.222222,0.118812
7,7.0,0.909091,0.938884,0.923747


In [49]:
final_weight_list

[array([1.6, 1.5, 1.5]),
 array([1.6, 1.5, 1.5]),
 array([1.2, 1.9, 0. ]),
 array([1.2, 1.9, 0. ]),
 array([1.2, 1.9, 0. ]),
 array([1.2, 1.9, 0. ]),
 array([1.2, 0. , 1.9]),
 array([0.1, 1.1, 1.9])]

# Test 

In [37]:
test_set = pd.read_csv('C:/Documents/Dental_Detection/data_csv/test_data_final.csv')
test_set.head()

,Unnamed: 0,file_path,anomaly_codes
0,0,C:/Documents/Dental_Detection/Segmented_Images...,7.0
1,1,C:/Documents/Dental_Detection/Segmented_Images...,0.0
2,2,C:/Documents/Dental_Detection/Segmented_Images...,7.0
3,3,C:/Documents/Dental_Detection/Segmented_Images...,7.0
4,4,C:/Documents/Dental_Detection/Segmented_Images...,7.0


In [38]:
test_set['anomaly_codes'].value_counts()

7.0    10668
0.0     1923
8.0      325
1.0      167
6.0      118
5.0       74
2.0       35
4.0       33
3.0       15
Name: anomaly_codes, dtype: int64

In [39]:
df_actuals_test = test_set['anomaly_codes']
df_actuals_dummied_test = pd.get_dummies(df_actuals_test.astype(int))

In [45]:
all_predictions_test = []
for i in range(len(models)):
    model = models[i]
    predictions = []

    for index, row in tqdm(test_set.iterrows(), total = test_set.shape[0]):
        image_path = row['file_path']
        img = cv2.imread(image_path)     # loads image from file
         # declaration of clahe
        clahe = cv2.createCLAHE(clipLimit=12.0, tileGridSize=(8,8))
        
        if((i == 0) | (i == 2)):
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = clahe.apply(img)
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

        img = cv2.resize(img, (224, 224))        # resize image to 224x224
        img = np.array(img, dtype=np.float32)    # change data type to float
        img = (img / 255.)                       # normalize colors to be 0-1
        img = img[:, :, (2, 1, 0)]               # reorder RGB
        img = torchvision.transforms.ToTensor()(img)  # convert to tensor
        target = row['anomaly_codes']

        image = img.to(device).unsqueeze(0)
        outputs = model(image)
        if i == 2:
            outputs = torch.sigmoid(outputs)
        outputs = outputs.detach().cpu()

        predictions += [outputs.squeeze().tolist()]
        df_predictions = pd.DataFrame(predictions)
    all_predictions_test += [df_predictions]

  0%|          | 0/13358 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\envs\dental_project_av\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 13358/13358 [02:56<00:00, 75.61it/s]


In [46]:
all_predictions_final_test = []
for i in range(len(models)):
    thresh = all_thresholds[i]
    preds = all_predictions_test[i]
    
    preds_final = preds.copy()
    for label in range(preds.shape[1]):
        preds_final[label] = pd.Series(preds[label] > thresh[label]).astype(int)
    
    all_predictions_final_test += [preds_final]

### Ensemble 1 Results - Test

In [47]:
final_test = ensemble_weighted(all_predictions_final_test, [1, 1, 1], vote_threshold = 1)
results_1_test = print_evaluation_results(vote_results = final_test, actuals = df_actuals_dummied_test)
results_1_test

,Class,Precision,Recall,F1
0,0.0,0.705660,0.583463,0.638770
1,1.0,0.414938,0.598802,0.490196
2,2.0,0.282353,0.685714,0.400000
3,3.0,0.003914,0.133333,0.007605
4,4.0,0.004505,0.151515,0.008749
5,5.0,0.019066,0.270270,0.035619
6,6.0,0.009259,0.703390,0.018278
7,7.0,0.810189,0.184852,0.301023


### Ensemble 2 Results - Test

In [49]:
final_2_test = ensemble_weighted(all_predictions_final_test, final_weights_valid)
results_2_test = print_evaluation_results(vote_results = final_2_test, actuals = df_actuals_dummied_test)
results_2_test

,Class,Precision,Recall,F1
0,0.0,0.696649,0.410816,0.516847
1,1.0,0.396226,0.377246,0.386503
2,2.0,0.352941,0.171429,0.230769
3,3.0,0.000000,0.000000,0.000000
4,4.0,0.000000,0.000000,0.000000
5,5.0,0.057143,0.027027,0.036697
6,6.0,0.010590,0.059322,0.017972
7,7.0,0.864962,0.928853,0.895769


### Ensemble 3 Results - Test

In [50]:
final_class_ensemble_test = ensemble_class(all_predictions_final_test, pd.DataFrame(final_weight_list))
results_3_test = print_evaluation_results(vote_results = final_class_ensemble_test, actuals = df_actuals_dummied_test)
results_3_test

,Class,Precision,Recall,F1
0,0.0,0.705660,0.583463,0.638770
1,1.0,0.438053,0.592814,0.503817
2,2.0,0.352941,0.171429,0.230769
3,3.0,0.000000,0.000000,0.000000
4,4.0,0.000000,0.000000,0.000000
5,5.0,0.057143,0.027027,0.036697
6,6.0,0.007463,0.025424,0.011538
7,7.0,0.893186,0.935133,0.913679
